<a href="https://colab.research.google.com/github/shatlykgurdov/3.1.2/blob/main/task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================
# Task 2 – Yield Curve Modeling (a–d)
# ============================================
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.interpolate import CubicSpline

# ------------------------------
# (a) Choice of government securities
# ------------------------------
maturities = np.array([0.5, 2, 5, 10, 20, 30])           # years
yields_pct = np.array([4.5, 4.7, 4.9, 5.1, 5.3, 5.4])    # percent

# ------------------------------
# (b) Nelson–Siegel model and fit (stable)
# ------------------------------
def _ns_terms_stable(tau, lam):
    """
    Stable computation of:
      term1 = (1 - e^{-x}) / x
      term2 = term1 - e^{-x}
    where x = tau / lam.
    Uses expm1 and a series expansion for small x to avoid overflow/underflow.
    """
    x = tau / lam
    term1 = np.empty_like(x, dtype=float)

    small = np.abs(x) < 1e-6
    # Stable for general x: (1 - e^{-x}) = -expm1(-x)
    term1[~small] = (-np.expm1(-x[~small])) / x[~small]
    # Series expansion for very small x
    xs = x[small]
    term1[small] = 1 - xs/2 + xs**2/6

    term2 = term1 - np.exp(-x)
    return term1, term2

def nelson_siegel(tau, beta0, beta1, beta2, lam):
    t1, t2 = _ns_terms_stable(tau, lam)
    return beta0 + beta1 * t1 + beta2 * t2

# Fit with bounds to keep λ positive and not tiny (prevents warnings)
p0 = [np.mean(yields_pct), -1.0, 1.0, 2.0]                     # initial guess
bounds = ([0.0,  -10.0, -10.0, 0.05], [20.0, 10.0, 10.0, 50.0]) # (lower, upper)

with np.errstate(over='ignore', invalid='ignore', under='ignore'):
    params, _ = curve_fit(nelson_siegel, maturities, yields_pct,
                          p0=p0, bounds=bounds, maxfev=20000)
beta0, beta1, beta2, lam = params

# Optional goodness-of-fit on observed points
ns_on_points = nelson_siegel(maturities, *params)
rmse = float(np.sqrt(np.mean((ns_on_points - yields_pct)**2)))

# ------------------------------
# (c) Cubic Spline interpolation
# ------------------------------
cs = CubicSpline(maturities, yields_pct, bc_type='not-a-knot')

# ------------------------------
# (d) Plot comparison
# ------------------------------
tau_fit = np.linspace(maturities.min(), maturities.max(), 400)
ns_fit = nelson_siegel(tau_fit, *params)
spline_fit = cs(tau_fit)

plt.figure(figsize=(9,6))
plt.scatter(maturities, yields_pct, label="Observed Yields", zorder=3)
plt.plot(tau_fit, ns_fit, label="Nelson–Siegel Fit", linewidth=2)
plt.plot(tau_fit, spline_fit, label="Cubic Spline Fit", linestyle="--", linewidth=2)
plt.xlabel("Maturity (Years)")
plt.ylabel("Yield (%)")
plt.title("Task 2: Yield Curve — Nelson–Siegel vs. Cubic Spline")
plt.legend()
plt.tight_layout()
plt.show()

print("Nelson–Siegel parameters:")
print(f"  β0 (level)     = {beta0:.6f}")
print(f"  β1 (slope)     = {beta1:.6f}")
print(f"  β2 (curvature) = {beta2:.6f}")
print(f"  λ  (decay)     = {lam:.6f}")
print(f"Nelson–Siegel RMSE on observed points = {rmse:.6f}")
